<a href="https://colab.research.google.com/github/Sthesha/Comp702-Project.-/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
main_path = "/root/nltk_data/corpora/wordnet.zip" # Specify the path
import os
if os.path.exists(main_path)== False:
  ! unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/
os.listdir('/root/nltk_data')
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline
# Below libraries are for feature representation using sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Below libraries are for similarity matrices using sklearn
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
import copy
from IPython.display import clear_output
import warnings
from re import sub
import seaborn as sns
from wordcloud import WordCloud
from sklearn import metrics
from sklearn.model_selection import train_test_split
from PIL import Image 
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from xgboost import XGBClassifier
import random
import nltk
from gensim.models import KeyedVectors
nltk.download('omw-1.4')
warnings.filterwarnings("ignore")

import re
from google.colab import auth
import os


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
os.listdir('/root/nltk_data')

['tokenizers', 'corpora', 'wordnet']

In [4]:

# !curl https://sdk.cloud.google.com | bash
# !gcloud init

In [5]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
main_path = "/content/mind-news-dataset.zip" # Specify the path

# Check whether the specified path exists or not
if os.path.exists(main_path)== False:
  ! kaggle datasets download arashnic/mind-news-dataset
  ! unzip /content/mind-news-dataset.zip


In [7]:
dtypes = {
    "News ID": "category",
    "Category": "category",
    "SubCategory": "category",
    "Title": "category",
    "Abstract": "category",
    "URL": "category",
    "Title Entities": "category",
    "Abstract Entities": "category"
}

In [8]:
news_data = pd.read_csv('/content/news.tsv/news.tsv', header=None, sep="\t")
news_data.columns = ['News ID', "Category", "SubCategory", "Title", "Abstract", "URL", "Title Entities" , "Abstract Entities"]


In [9]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51282 entries, 0 to 51281
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   News ID            51282 non-null  object
 1   Category           51282 non-null  object
 2   SubCategory        51282 non-null  object
 3   Title              51282 non-null  object
 4   Abstract           48616 non-null  object
 5   URL                51282 non-null  object
 6   Title Entities     51279 non-null  object
 7   Abstract Entities  51278 non-null  object
dtypes: object(8)
memory usage: 3.1+ MB


In [10]:
news_data = news_data.astype(dtypes)

In [11]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51282 entries, 0 to 51281
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   News ID            51282 non-null  category
 1   Category           51282 non-null  category
 2   SubCategory        51282 non-null  category
 3   Title              51282 non-null  category
 4   Abstract           48616 non-null  category
 5   URL                51282 non-null  category
 6   Title Entities     51279 non-null  category
 7   Abstract Entities  51278 non-null  category
dtypes: category(8)
memory usage: 11.5 MB


In [12]:
def reduce_mem_usage(news_data, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = news_data.memory_usage().sum() / 1024**2
    for col in news_data.columns:
        col_type = news_data[col].dtypes
        if col_type in numerics:
            c_min = news_data[col].min()
            c_max = news_data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    news_data[col] = news_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    news_data[col] = news_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    news_data[col] = news_data[col].astype(np.int32)
                elif news_data > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    news_data[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    news_datanews_data[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    news_datanews_datanews_data[col] = news_data[col].astype(np.float32)
                else:
                    news_datanews_datanews_datanews_data[col] = df[col].astype(np.float64)

    end_mem = news_data.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return news_data

In [13]:
news_data = reduce_mem_usage(news_data)

Memory usage after optimization is: 11.46 MB
Decreased by 0.0%


In [14]:
news_data.head()

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [15]:
news_data.drop(['URL', 'Title Entities', 'Abstract Entities'], axis=1)

,News ID,Category,SubCategory,Title,Abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."
...,...,...,...,...,...
51277,N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...
51278,N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died fro...,NaN
51279,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b..."
51280,N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ..."


In [16]:
users_behavior = pd.read_csv('/content/MINDsmall_train/behaviors.tsv', header=None, sep="\t")
users_behavior.columns = ["Impression", "User ID", "Time", "History", "Impressions"]

print('the number of articles before processing :',len(users_behavior))
users_behavior=users_behavior.dropna().reset_index(drop=True)
print('The number of articles after processing :',len(users_behavior))
users_behavior.head(10)


the number of articles before processing : 156965
The number of articles after processing : 153727


,Impression,User ID,Time,History,Impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...
5,6,U19739,11/11/2019 6:52:13 PM,N39074 N14343 N32607 N32320 N22007 N442 N19001...,N21119-1 N53696-0 N33619-1 N25722-0 N2869-0
6,7,U8355,11/11/2019 12:22:09 PM,N8419 N15771 N1431 N5888 N18663 N24123 N22130 ...,N51346-0 N33848-0 N15132-0 N10688-0 N6342-0 N6...
7,8,U46596,11/12/2019 10:29:36 PM,N47438 N20950 N21317 N5469,N7821-0 N24898-0 N12029-0 N13579-0 N42977-0 N3...
8,9,U79199,11/13/2019 10:13:02 AM,N37083 N459 N29499 N38118 N37378 N24691 N27235...,N51048-1 N64094-0 N13907-0 N39010-0
9,10,U53231,11/11/2019 11:28:11 AM,N58936 N15919 N11917 N2153 N55312 N13008 N4142...,N53585-1 N55689-0


In [17]:

print('the number of articles before processing :',len(news_data))
news_data.drop_duplicates(subset=["Abstract"],inplace=True)
print('The number of articles after processing :',len(news_data))

the number of articles before processing : 51282
The number of articles after processing : 47310


In [18]:
# #New dataframe
# #having the user and the news read and those which were not either one or zero
# user_history = pd.DataFrame(columns = ["user_id", "history", "verdict"])
# for index, row in users_behavior.iterrows():
#   hist = row["History"]
#   user_id = row["User ID"]
#   impressions = row["Impressions"]
#   impressions = list(impressions.split(" "))
#   hist = list(hist.split(" "))
#   for imp in impressions:
#     user_history.loc[len(user_history.index)] = [user_id, imp, "1"]
# user_history.head()


In [19]:
# #create new user ID 
# user_history = pd.DataFrame(columns = ["user_id", "history"])
# for index, row in users_behavior.iterrows():
#   hist = row["History"]
#   user_id = row["User ID"]
#   impressions = row["Impressions"]
#   impressions = list(impressions.split(" "))  
#   for imp in impressions:
#     if (imp[-1]=="1"):
#       imp = imp[:-2]
#       hist = hist + " " + imp
#   user_history.loc[len(user_history.index)] = [user_id, hist] 
      



        

In [20]:
news_data.isna().sum()
news_data.isna().sum()
news_data.dropna(inplace=True)

In [21]:
print('The number of articles after processing :',len(news_data))

The number of articles after processing : 47305


In [22]:
def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    processedSentence = []
    word_tokens = word_tokenize(sentence)
    for i in word_tokens:
        if i not in stop_words:
            processedSentence.append(i)
    return processedSentence

In [23]:
lemmatizer = WordNetLemmatizer() 
alphaPattern = "[^a-zA-Z0-9<>]"
numerical = comment = "[0-9]+"

# Making a function to lemmatize all the words
lemmatizer = WordNetLemmatizer() 

def lemmatize_column(data, col_name):
        processedSentence = []
        sentences = data[col_name]
        for sentence in sentences:
            sentence = sentence.lower()
            sentence = re.sub(alphaPattern, " ", sentence)
            sentence = re.sub(numerical, " ", sentence)

            words = remove_stopwords(sentence)
            holder = []
            for word in words:
                word =lemmatizer.lemmatize(word)
                if(len(word)>2):
                  holder.append(word)
            processedSentence.append(' '.join(holder))
        data[col_name]=processedSentence
        return data

In [24]:
lemmatize_column(news_data, "Abstract")

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...",shop notebook jacket royal live without,https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,seemingly harmless habit holding back keeping ...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,ivan molchanets peeked parapet sand bag front ...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,felt like fraud nba wife help fact nearly dest...,https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...",seem harmless good reason ignore post get rid ...,https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
...,...,...,...,...,...,...,...,...
51274,N43432,news,newsus,US Forest Service shuts down vandalized Georgi...,gainesville forest service say vandalism forci...,https://assets.msn.com/labs/mind/BBWzDfU.html,"[{""Label"": ""Georgia (U.S. state)"", ""Type"": ""G""...","[{""Label"": ""Georgia (U.S. state)"", ""Type"": ""G""..."
51275,N17258,news,newsscienceandtechnology,Realme takes chunk of India mobile market as S...,percent phone shipped year year,https://assets.msn.com/labs/mind/BBWzDfx.html,"[{""Label"": ""Realme"", ""Type"": ""N"", ""WikidataId""...",[]
51276,N23858,sports,golf,Young Northeast Florida fans flock to U.S. wom...,woman national soccer team arrived jacksonvill...,https://assets.msn.com/labs/mind/BBWzQB8.html,"[{""Label"": ""First Coast"", ""Type"": ""G"", ""Wikida...","[{""Label"": ""United States women's national soc..."
51279,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,sometimes happens sideline even important happ...,https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]


In [43]:
news_data = news_data.reset_index()


In [44]:
news_data["Abstract"]

0                  shop notebook jacket royal live without
1        seemingly harmless habit holding back keeping ...
2        ivan molchanets peeked parapet sand bag front ...
3        felt like fraud nba wife help fact nearly dest...
4        seem harmless good reason ignore post get rid ...
                               ...                        
47300    gainesville forest service say vandalism forci...
47301                      percent phone shipped year year
47302    woman national soccer team arrived jacksonvill...
47303    sometimes happens sideline even important happ...
47304         mark jeremiah casey excited postgame podcast
Name: Abstract, Length: 47305, dtype: object

In [48]:
news_data["Abstract"][37203]


'year old bison named beefcake version snow angel henry vila zoo madison wisconsin november video shared zoo taken zookeeper taylor wood beefcake roll large body around snow kick leg air stand shake snow mane zoo post accompanying facebook video said polar bear one enjoying snow beefcake one happy boy today madison part southern wisconsin hit six inch snow november local medium reported road interstate system southern wisconsin snow covered department transportation also reported number accident spinouts milwaukee area interstate credit henry vila zoo via storyful'

In [27]:
corpus = []
for words in news_data["Abstract"]:
    corpus.append(words.split())
print(corpus[0])

['shop', 'notebook', 'jacket', 'royal', 'live', 'without']


In [28]:
main_path = "/content/googlenewsvectorsnegative300.zip" # Specify the path

# Check whether the specified path exists or not
if os.path.exists(main_path)== False:
  ! kaggle datasets download leadbest/googlenewsvectorsnegative300
  ! unzip /content/googlenewsvectorsnegative300.zip




In [29]:
# Load Word2Vec model (trained on an enormous Google corpus)
google_word2vec = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary = True) 
# Check dimension of word vectors
print(google_word2vec)

KeyedVectors<vector_size=300, 3000000 keys>


In [30]:
google_word2vec.vector_size

300

In [31]:
pip install --upgrade gensim


In [32]:

# Training our corpus with Google Pretrained Model
#create word to vector model 
model = Word2Vec(vector_size = 300, min_count = 1)
model.build_vocab(corpus)
model.wv.vectors_lockf = np.ones(len(model.wv))

model.wv.intersect_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True, lockf=1.0)
print(model.corpus_count)
model.train(corpus, total_examples=model.corpus_count, epochs = 5)

47305


(4981698, 5064675)

In [33]:
#Building TFIDF model and calculate TFIDF score

tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 5)
tfidf.fit(news_data['Abstract'])

# Getting the words from the TF-IDF model

tfidf_list = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))
tfidf_feature = tfidf.get_feature_names() # tfidf words/col-names

In [34]:
news_data = news_data.reset_index()


In [35]:
vocabulary = model.wv.key_to_index
w2v_headline = []
for i in news_data["Abstract"]:
    w2Vec_word = np.zeros(300, dtype="float32")
    for word in i.split():
        if word in vocabulary:
          w2Vec_word = np.add(w2Vec_word, model.wv[word])
    w2Vec_word = np.divide(w2Vec_word, len(i.split()))
    w2v_headline.append(w2Vec_word)

w2v_headline = np.array(w2v_headline)

# print(w2v_headline[11].reshape(1,-1))
print(w2v_headline)
print(np.isfinite(w2v_headline).all())
print(np.all(np.isfinite(w2v_headline)))

w2v_headline =np.nan_to_num(w2v_headline)



[[ 0.09006446  0.0497372  -0.18740945 ...  0.04982223  0.14883347
  -0.08128146]
 [ 0.07908094  0.22546615 -0.05977982 ...  0.0257273   0.01558878
  -0.02480219]
 [ 0.06518795  0.14794789  0.07906339 ... -0.11218343  0.02746166
   0.07702471]
 ...
 [ 0.01229006  0.1440721   0.11162078 ... -0.12229433 -0.03502036
  -0.12493409]
 [ 0.01642523  0.10055196  0.19363576 ... -0.121034   -0.0276818
  -0.07087583]
 [-0.06199593  0.10866774 -0.05136494 ... -0.18859093 -0.13234614
  -0.11191014]]
False
False


In [47]:
def avg_w2v_based_model(row_index, num_similar_items):
    couple_dist = pairwise_distances(w2v_headline, w2v_headline[row_index].reshape(1,-1))
    indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
    print(indices[0])
    df = pd.DataFrame({"ID ": news_data['News ID'][indices].values, 'Category': news_data['Category'][indices].values,
               'headline':news_data['Abstract'][indices].values,
                'Euclidean similarity with the queried article': couple_dist[indices].ravel()})
    print("="*30,"Queried article details","="*30)

    print('headline : ',news_data['Abstract'][indices[0]], "category : ", news_data['Category'][indices[0]])
    print("\n","="*25,"Recommended articles : ","="*23)
    #return df.iloc[1:,1]
    return df.iloc[1:,]

avg_w2v_based_model(47300, 11)

47300
============================== Queried article details ==============================
headline :  gainesville forest service say vandalism forcing shut popular fall leaf viewing destination georgia agency posted photo facebook friday showing wooden platform covered white spray paint overlooking mountainous north georgia landscape chattahoochee oconee national forest category :  news

 ========================= Recommended articles :  =======================


,ID,Category,headline,Euclidean similarity with the queried article
1,N3299,weather,snowstorm moving colorado tuesday october left...,1.523503
2,N43859,weather,new hampshire resident awoke friday november f...,1.623018
3,N33590,video,year old bison named beefcake version snow ang...,1.641635
4,N24813,weather,southeast new brunswick canada blanketed snow ...,1.678059
5,N35962,weather,eight inch snow fell across madison wisconsin ...,1.685688
6,N17800,weather,resident experienced first substantial snow se...,1.686617
7,N60334,weather,potent snowstorm moving colorado tuesday octob...,1.727900
8,N11046,weather,aftermath tornado swept garland texas northeas...,1.728107
9,N7747,weather,blustery snow moved santa new mexico october f...,1.730612
10,N60296,weather,resident western pennsylvania got first signif...,1.735630


In [37]:
# Building TF-IDF Word2Vec

# Storing the TFIDF Word2Vec embeddings
tfidf_vectors = []; 
line = 0;
# for each book description
for desc in corpus:
  # Word vectors are of zero length (Used 300 dimensions)
    sent_vec = np.zeros(300)
    # num of words with a valid vector in the book descriptionAbstract
    weight_sum = 0; 
    # for each word in the book description
    
    for word in desc: 
        if word in model.wv.key_to_index and word in tfidf_feature:
            vec = model.wv[word]
            tf_idf = tfidf_list[word] * (desc.count(word) / len(desc))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_vectors.append(sent_vec)
    line += 1

In [38]:

def recommendations(abstract):
    # finding cosine similarity for the vectors
    print("yes")

    cosine_similarities = cosine_similarity(tfidf_vectors, tfidf_vectors)

    # taking the title and book image link and store in new data frame called books
    books = news_data[['Abstract', 'Title', 'Category']]
    # Reverse mapping of the index
    indices = pd.Series(news_data.index, index=news_data['Abstract']).drop_duplicates()

    idx = indices[abstract]
    print(" yes ")
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    print(sim_scores)
    book_indices = [i[0] for i in sim_scores]
    print(book_indices)

    df = pd.DataFrame({"ID ": news_data['News ID'][book_indices].values, 'Category': news_data['Category'][book_indices].values, 'headline':news_data['Abstract'][book_indices].values })
    print("="*30,"Queried article details","="*30)
    print('headline : ',news_data['Abstract'][50], "category : ", news_data['Category'][book_indices[0]])
    print("\n","="*25,"Recommended articles : ","="*23)
    df.info()
    print(df)

        

In [46]:
recommendations("gainesville forest service say vandalism forcing shut popular fall leaf viewing destination georgia agency posted photo facebook friday showing wooden platform covered white spray paint overlooking mountainous north georgia landscape chattahoochee oconee national forest")

yes
 yes 
[(40197, 0.879060576563512), (42217, 0.8761379668913889), (34006, 0.8663210970049992), (37203, 0.8630751766912272), (6656, 0.8621257582533111), (9225, 0.8611346086141614), (31771, 0.8595566412069511), (40498, 0.8589806561101598), (6200, 0.8554894409028518), (6792, 0.8553808166663661)]
[40197, 42217, 34006, 37203, 6656, 9225, 31771, 40498, 6200, 6792]
============================== Queried article details ==============================
headline :  welcome house please drop bag right category :  weather

 ========================= Recommended articles :  =======================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ID        10 non-null     category
 1   Category  10 non-null     category
 2   headline  10 non-null     object  
dtypes: category(2), object(1)
memory usage: 2.4+ MB
      ID  Category                                   